# Data Preprocessing

In [1]:
import numpy as np # Linear algebra
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt  # Matlab-style plotting
# Make sure plot shows immediately
%matplotlib inline 
import seaborn as sns # Library for plotting
color = sns.color_palette()
sns.set_style('darkgrid')

# Strategy A

In [63]:
df_claim = pd.read_csv("./data/Claims.csv")
df_member = pd.read_csv("./data/Members.csv")
df_lab = pd.read_csv("./data/LabCount.csv")
df_drug = pd.read_csv("./data/DrugCount.csv")


## Member

In [3]:
age_list = list(df_member["AgeAtFirstClaim"].value_counts().index)
age_list.sort()
age_list

['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']

In [4]:
mean_age_list = [ 5, 15, 25, 35, 45, 55, 65, 75, 85]

In [5]:
for age, mean_age in zip(age_list, mean_age_list):
    df_member.loc[df_member["AgeAtFirstClaim"] == age, "AgeAtFirstClaim"] = mean_age

In [6]:
df_member

,MemberID,AgeAtFirstClaim,Sex
0,14723353,75,M
1,75706636,75,M
2,17320609,75,M
3,69690888,45,M
4,33004608,5,M
...,...,...,...
112995,99711514,45,F
112996,31690877,55,F
112997,9519985,35,F
112998,92806272,55,F


In [7]:
df_member.to_csv("./processed/Members.csv", index=False)

## ClaimCount

In [64]:
df_claim_y1 = df_claim[df_claim["Year"] == "Y1"]
print(df_claim_y1.shape)
df_claim_y2 = df_claim[df_claim["Year"] == "Y2"]
print(df_claim_y2.shape)
df_claim_y12 = df_claim[(df_claim["Year"] == "Y1") | (df_claim["Year"] == "Y2")]
print(df_claim_y12.shape)

(865689, 14)
(898872, 14)
(1764561, 14)


In [9]:
count_y1 = df_claim_y1["MemberID"].value_counts()
count_y2 = df_claim_y2["MemberID"].value_counts()
count_y12 = df_claim_y12["MemberID"].value_counts()

In [10]:
df_claim_count_y1 = pd.DataFrame({ "MemberID": list(count_y1.index), "ClaimCount": list(count_y1)})
df_claim_count_y2 = pd.DataFrame({ "MemberID": list(count_y2.index), "ClaimCount": list(count_y2)})
df_claim_count_y12 = pd.DataFrame({ "MemberID": list(count_y12.index), "ClaimCount": list(count_y12)})

In [11]:
df_claim_count_y1.to_csv("./processed/ClaimCount_Y1.csv", index=False)
df_claim_count_y2.to_csv("./processed/ClaimCount_Y2.csv", index=False)
df_claim_count_y12.to_csv("./processed/ClaimCount_Y12.csv", index=False)

## Provider

In [12]:
pvd_count_y1 = df_claim_y1.groupby("MemberID")["ProviderID"].nunique(dropna=True)
pvd_count_y2 = df_claim_y2.groupby("MemberID")["ProviderID"].nunique(dropna=True)
pvd_count_y12 = df_claim_y12.groupby("MemberID")["ProviderID"].nunique(dropna=True)

In [13]:
df_pvd_count_y1 = pd.DataFrame({ "MemberID": pvd_count_y1.index, "ProviderID": pvd_count_y1})
df_pvd_count_y2 = pd.DataFrame({ "MemberID": pvd_count_y2.index, "ProviderID": pvd_count_y2})
df_pvd_count_y12 = pd.DataFrame({ "MemberID": pvd_count_y12.index, "ProviderID": pvd_count_y12})

In [14]:
df_pvd_count_y1.to_csv("./processed/ProviderID_Y1.csv", index=False)
df_pvd_count_y2.to_csv("./processed/ProviderID_Y2.csv", index=False)
df_pvd_count_y12.to_csv("./processed/ProviderID_Y12.csv", index=False)

## Vendor

In [15]:
v_count_y1 = df_claim_y1.groupby("MemberID")["Vendor"].nunique(dropna=True)
v_count_y2 = df_claim_y2.groupby("MemberID")["Vendor"].nunique(dropna=True)
v_count_y12 = df_claim_y12.groupby("MemberID")["Vendor"].nunique(dropna=True)

In [16]:
df_v_count_y1 = pd.DataFrame({"MemberID": v_count_y1.index, "Vendor": v_count_y1})
df_v_count_y2 = pd.DataFrame({"MemberID": v_count_y2.index, "Vendor": v_count_y2})
df_v_count_y12 = pd.DataFrame({"MemberID": v_count_y12.index, "Vendor": v_count_y12})

In [17]:
df_v_count_y1.to_csv("./processed/Vendor_Y1.csv", index=False)
df_v_count_y2.to_csv("./processed/Vendor_Y2.csv", index=False)
df_v_count_y12.to_csv("./processed/Vendor_Y12.csv", index=False)

## PCP

In [18]:
pcp_count_y1 = df_claim_y1.groupby("MemberID")["PCP"].nunique(dropna=True)
pcp_count_y2 = df_claim_y2.groupby("MemberID")["PCP"].nunique(dropna=True)
pcp_count_y12 = df_claim_y12.groupby("MemberID")["PCP"].nunique(dropna=True)

In [19]:
df_pcp_count_y1 = pd.DataFrame({ "MemberID": pcp_count_y1.index, "PCP": pcp_count_y1})
df_pcp_count_y2 = pd.DataFrame({ "MemberID": pcp_count_y2.index, "PCP": pcp_count_y2})
df_pcp_count_y12 = pd.DataFrame({ "MemberID": pcp_count_y12.index, "PCP": pcp_count_y12})

In [20]:
df_pcp_count_y1.to_csv("./processed/PCP_Y1.csv", index=False)
df_pcp_count_y2.to_csv("./processed/PCP_Y2.csv", index=False)
df_pcp_count_y12.to_csv("./processed/PCP_Y12.csv", index=False)

## Specialty

In [21]:
def one_hot_count(index_col, count_value_col, df, one_hot_cols):
    # get unique index
    index_list = list(df[index_col].unique())
    # create df
    result_df = pd.DataFrame(index=index_list)
    result_df.index.name = index_col
    # fill one hot columns
    for col in one_hot_cols:
        col_name = "{}Count_{}".format(count_value_col, col)
        result_df[col_name] = 0
    # query and fill count
    query = df.groupby([index_col])[count_value_col].value_counts(dropna=False)
    i = 0
    length = len(index_list)
    for index in index_list:
        for col in one_hot_cols:
            col_name = "{}Count_{}".format(count_value_col, col)
            try:
                result_df.loc[index][col_name] = query.loc[(index, col)]
            except:
                continue

        if i % 10000 == 0:
            print("Processed: {}/{}".format(i, length))
        i += 1
    print("Processed: {}/{} DONE".format(i, length))
    return result_df

In [22]:
spe_unique_y1 = list(df_claim_y1["Specialty"].unique())
spe_unique_y2 = list(df_claim_y2["Specialty"].unique())
spe_unique_y12 = list(df_claim_y12["Specialty"].unique())

In [23]:
%%time
df_spe_count_y1 = one_hot_count("MemberID", "Specialty", df_claim_y1, spe_unique_y1)
df_spe_count_y2 = one_hot_count("MemberID", "Specialty", df_claim_y2, spe_unique_y2)
df_spe_count_y12 = one_hot_count("MemberID", "Specialty", df_claim_y12, spe_unique_y12)

Processed: 0/76038
Processed: 10000/76038
Processed: 20000/76038
Processed: 30000/76038
Processed: 40000/76038
Processed: 50000/76038
Processed: 60000/76038
Processed: 70000/76038
Processed: 76038/76038 DONE
Processed: 0/71435
Processed: 10000/71435
Processed: 20000/71435
Processed: 30000/71435
Processed: 40000/71435
Processed: 50000/71435
Processed: 60000/71435
Processed: 70000/71435
Processed: 71435/71435 DONE
Processed: 0/95507
Processed: 10000/95507
Processed: 20000/95507
Processed: 30000/95507
Processed: 40000/95507
Processed: 50000/95507
Processed: 60000/95507
Processed: 70000/95507
Processed: 80000/95507
Processed: 90000/95507
Processed: 95507/95507 DONE
CPU times: user 6min 54s, sys: 7.43 s, total: 7min 2s
Wall time: 10min 37s


In [24]:
df_spe_count_y1.to_csv("./processed/SpecialtyCount_Y1.csv")
df_spe_count_y2.to_csv("./processed/SpecialtyCount_Y2.csv")
df_spe_count_y12.to_csv("./processed/SpecialtyCount_Y12.csv")

## PlaceSvc

In [25]:
psvc_unique_y1 = list(df_claim_y1["PlaceSvc"].unique())
psvc_unique_y2 = list(df_claim_y2["PlaceSvc"].unique())
psvc_unique_y12 = list(df_claim_y12["PlaceSvc"].unique())

In [26]:
%%time
df_psvc_count_y1 = one_hot_count("MemberID", "PlaceSvc", df_claim_y1, psvc_unique_y1)
df_psvc_count_y2 = one_hot_count("MemberID", "PlaceSvc", df_claim_y2, psvc_unique_y2)
df_psvc_count_y12 = one_hot_count("MemberID", "PlaceSvc", df_claim_y12, psvc_unique_y12)

Processed: 0/76038
Processed: 10000/76038
Processed: 20000/76038
Processed: 30000/76038
Processed: 40000/76038
Processed: 50000/76038
Processed: 60000/76038
Processed: 70000/76038
Processed: 76038/76038 DONE
Processed: 0/71435
Processed: 10000/71435
Processed: 20000/71435
Processed: 30000/71435
Processed: 40000/71435
Processed: 50000/71435
Processed: 60000/71435
Processed: 70000/71435
Processed: 71435/71435 DONE
Processed: 0/95507
Processed: 10000/95507
Processed: 20000/95507
Processed: 30000/95507
Processed: 40000/95507
Processed: 50000/95507
Processed: 60000/95507
Processed: 70000/95507
Processed: 80000/95507
Processed: 90000/95507
Processed: 95507/95507 DONE
CPU times: user 5min 19s, sys: 6.46 s, total: 5min 26s
Wall time: 8min 28s


In [27]:
df_psvc_count_y1.to_csv("./processed/PlaceSvcCount_Y1.csv")
df_psvc_count_y2.to_csv("./processed/PlaceSvcCount_Y2.csv")
df_psvc_count_y12.to_csv("./processed/PlaceSvcCount_Y12.csv")

## PayDelay

In [28]:
df_claim_y1["PayDelay"].replace({'162+': '162'}, inplace=True)
df_claim_y2["PayDelay"].replace({'162+': '162'}, inplace=True)
df_claim_y12["PayDelay"].replace({'162+': '162'}, inplace=True)

In [30]:
df_claim_y1["PayDelay"] = df_claim_y1["PayDelay"].astype("int32")
df_claim_y2["PayDelay"] = df_claim_y2["PayDelay"].astype("int32")
df_claim_y12["PayDelay"] = df_claim_y12["PayDelay"].astype("int32")

In [33]:
pdl_min_y1 = df_claim_y1.groupby(["MemberID"])["PayDelay"].min()
pdl_max_y1 = df_claim_y1.groupby(["MemberID"])["PayDelay"].max()
pdl_avg_y1 = df_claim_y1.groupby(["MemberID"])["PayDelay"].mean()
pdl_std_y1 = df_claim_y1.groupby(["MemberID"])["PayDelay"].std()
pdl_sum_y1 = df_claim_y1.groupby(["MemberID"])["PayDelay"].sum()

pdl_min_y2 = df_claim_y2.groupby(["MemberID"])["PayDelay"].min()
pdl_max_y2 = df_claim_y2.groupby(["MemberID"])["PayDelay"].max()
pdl_avg_y2 = df_claim_y2.groupby(["MemberID"])["PayDelay"].mean()
pdl_std_y2 = df_claim_y2.groupby(["MemberID"])["PayDelay"].std()
pdl_sum_y2 = df_claim_y2.groupby(["MemberID"])["PayDelay"].sum()

pdl_min_y12 = df_claim_y12.groupby(["MemberID"])["PayDelay"].min()
pdl_max_y12 = df_claim_y12.groupby(["MemberID"])["PayDelay"].max()
pdl_avg_y12 = df_claim_y12.groupby(["MemberID"])["PayDelay"].mean()
pdl_std_y12 = df_claim_y12.groupby(["MemberID"])["PayDelay"].std()
pdl_sum_y12 = df_claim_y12.groupby(["MemberID"])["PayDelay"].sum()

In [45]:
pdl_sum_y12

MemberID
4             43
210         1028
3197         640
3457          63
3713         562
            ... 
99996214      19
99997485     130
99997895     539
99998627     526
99998824     343
Name: PayDelay, Length: 95507, dtype: int32

In [46]:
df_pdl_y1 = pd.DataFrame({ 
    "MemberID": pdl_min_y1.index, 
    "PayDelayMin": pdl_min_y1,
    "PayDelayMax": pdl_max_y1,
    "PayDelayAvg": pdl_avg_y1,
    "PayDelayStd": pdl_std_y1,
    "PayDelaySum": pdl_sum_y1
})

df_pdl_y2 = pd.DataFrame({ 
    "MemberID": pdl_min_y2.index, 
    "PayDelayMin": pdl_min_y2,
    "PayDelayMax": pdl_max_y2,
    "PayDelayAvg": pdl_avg_y2,
    "PayDelayStd": pdl_std_y2,
    "PayDelaySum": pdl_sum_y2
})

df_pdl_y12 = pd.DataFrame({ 
    "MemberID": pdl_min_y12.index, 
    "PayDelayMin": pdl_min_y12,
    "PayDelayMax": pdl_max_y12,
    "PayDelayAvg": pdl_avg_y12,
    "PayDelayStd": pdl_std_y12,
    "PayDelaySum": pdl_sum_y12
})

In [49]:
df_pdl_y1.to_csv("./processed/PayDelay_Y1.csv", index=False)
df_pdl_y2.to_csv("./processed/PayDelay_Y2.csv", index=False)
df_pdl_y12.to_csv("./processed/PayDelay_Y12.csv", index=False)

## LengthOfStay

In [71]:
%%time
# convert los to day
los_value = [ "1 day", "2 days", "3 days", "4 days", "5 days", "6 days", "1- 2 weeks", "2- 4 weeks", "4- 8 weeks", "8- 12 weeks", "12- 26 weeks", "26+ weeks"]
los_replace = [ 1, 2, 3, 4, 5, 6, 11, 21, 42, 84, 133, 182]

for i, j in zip(los_value, los_replace):
    df_claim_y1["LengthOfStay"].replace({i: j}, inplace=True)
    df_claim_y2["LengthOfStay"].replace({i: j}, inplace=True)
    df_claim_y12["LengthOfStay"].replace({i: j}, inplace=True)

df_claim_y1["LengthOfStay"] = df_claim_y1["LengthOfStay"].astype("float64")
df_claim_y2["LengthOfStay"] = df_claim_y2["LengthOfStay"].astype("float64")
df_claim_y12["LengthOfStay"] = df_claim_y12["LengthOfStay"].astype("float64")

CPU times: user 1.15 s, sys: 158 ms, total: 1.31 s
Wall time: 1.55 s


In [151]:
def process_los(df):
    los_min = df.groupby(["MemberID"])["LengthOfStay"].min()
    los_max = df.groupby(["MemberID"])["LengthOfStay"].max()
    los_avg = df.groupby(["MemberID"])["LengthOfStay"].mean()
    los_std = df.groupby(["MemberID"])["LengthOfStay"].std()
    los_sum = df.groupby(["MemberID"])["LengthOfStay"].sum()
    los_nan = df[df["LengthOfStay"].isna()].groupby(["MemberID"])["MemberID"].count()

    return pd.concat([ 
        pd.DataFrame(np.array(los_min).T, columns=["LengthOfStayMin"], index=los_min.index),
        pd.DataFrame(np.array(los_max).T, columns=["LengthOfStayMax"], index=los_max.index),
        pd.DataFrame(np.array(los_avg).T, columns=["LengthOfStayAvg"], index=los_avg.index),
        pd.DataFrame(np.array(los_std).T, columns=["LengthOfStayStd"], index=los_std.index),
        pd.DataFrame(np.array(los_sum).T, columns=["LengthOfStaySum"], index=los_sum.index),
        pd.DataFrame(np.array(los_nan).T, columns=["LengthOfStayCountNan"], index=los_nan.index)
    ], axis=1)

In [152]:
df_los_y1 = process_los(df_claim_y1)
df_los_y2 = process_los(df_claim_y2)
df_los_y12 = process_los(df_claim_y12)

In [154]:
df_los_y1.to_csv("./processed/LengthOfStay_Y1.csv")
df_los_y2.to_csv("./processed/LengthOfStay_Y2.csv")
df_los_y12.to_csv("./processed/LengthOfStay_Y12.csv")

## DSFS

In [159]:
%%time
# convert to month
dsfs_value = [ '0- 1 month', '1- 2 months', '2- 3 months', '3- 4 months', '4- 5 months', '5- 6 months', '6- 7 months', '7- 8 months', '8- 9 months', '9-10 months', '10-11 months', '11-12 months']
dsfs_replace = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ]

try:
    for i, j in zip(dsfs_value, dsfs_replace):
        df_claim_y1["DSFS"].replace({i: j}, inplace=True)
        df_claim_y2["DSFS"].replace({i: j}, inplace=True)
        df_claim_y12["DSFS"].replace({i: j}, inplace=True)


    df_claim_y1["DSFS"] = df_claim_y1["DSFS"].astype("float64")
    df_claim_y2["DSFS"] = df_claim_y2["DSFS"].astype("float64")
    df_claim_y12["DSFS"] = df_claim_y12["DSFS"].astype("float64")
except: 
    pass

CPU times: user 263 µs, sys: 14 µs, total: 277 µs
Wall time: 411 µs


In [163]:
def process_dsfs(df):
    dsfs_min = df.groupby(["MemberID"])["DSFS"].min()
    dsfs_max = df.groupby(["MemberID"])["DSFS"].max()

    return pd.concat([
        pd.DataFrame(np.array(dsfs_min).T, columns=["DSFSMin"], index=dsfs_min.index),
        pd.DataFrame(np.array(dsfs_max).T, columns=["DSFSMax"], index=dsfs_max.index),
    ], axis=1)

In [164]:
%%time
df_dsfs_y1 = process_dsfs(df_claim_y1)
df_dsfs_y2 = process_dsfs(df_claim_y2)
df_dsfs_y12 = process_dsfs(df_claim_y12)

CPU times: user 535 ms, sys: 73.5 ms, total: 608 ms
Wall time: 1.24 s


In [169]:
df_dsfs_y1.to_csv("./processed/DSFS_Y1.csv")
df_dsfs_y2.to_csv("./processed/DSFS_Y2.csv")
df_dsfs_y12.to_csv("./processed/DSFS_Y12.csv")

## CharlsonIndex

In [170]:
%%time
# convert to month
ch_value = [ '0', '1-2', '3-4', '5+' ]
ch_replace = [ 0, 2, 4, 6 ]

try:
    for i, j in zip(ch_value, ch_replace):
        df_claim_y1["CharlsonIndex"].replace({i: j}, inplace=True)
        df_claim_y2["CharlsonIndex"].replace({i: j}, inplace=True)
        df_claim_y12["CharlsonIndex"].replace({i: j}, inplace=True)


    df_claim_y1["CharlsonIndex"] = df_claim_y1["CharlsonIndex"].astype("float64")
    df_claim_y2["CharlsonIndex"] = df_claim_y2["CharlsonIndex"].astype("float64")
    df_claim_y12["CharlsonIndex"] = df_claim_y12["CharlsonIndex"].astype("float64")
except: 
    pass

CPU times: user 1.39 s, sys: 477 ms, total: 1.87 s
Wall time: 3.02 s


In [181]:
def process_charlsonindex(df):
    ch_min = df.groupby(["MemberID"])["CharlsonIndex"].min()
    ch_max = df.groupby(["MemberID"])["CharlsonIndex"].max()
    ch_avg = df.groupby(["MemberID"])["CharlsonIndex"].mean()

    return pd.concat([
        pd.DataFrame(np.array(ch_min).T, columns=["CharlsonIndexMin"], index=ch_min.index),
        pd.DataFrame(np.array(ch_max).T, columns=["CharlsonIndexMax"], index=ch_max.index),
        pd.DataFrame(np.array(ch_avg).T, columns=["CharlsonIndexAvg"], index=ch_avg.index),
    ], axis=1)

In [182]:
df_ch_y1 = process_charlsonindex(df_claim_y1)
df_ch_y2 = process_charlsonindex(df_claim_y2)
df_ch_y12 = process_charlsonindex(df_claim_y12)

In [185]:
df_ch_y1.to_csv("./processed/CharlsonIndex_Y1.csv")
df_ch_y2.to_csv("./processed/CharlsonIndex_Y2.csv")
df_ch_y12.to_csv("./processed/CharlsonIndex_Y12.csv")

## PrimaryConditionGroup

In [186]:
pcg_unique_y1 = list(df_claim_y1["PrimaryConditionGroup"].unique())
pcg_unique_y2 = list(df_claim_y2["PrimaryConditionGroup"].unique())
pcg_unique_y12 = list(df_claim_y12["PrimaryConditionGroup"].unique())

In [187]:
%%time
df_pcg_count_y1 = one_hot_count("MemberID", "PrimaryConditionGroup", df_claim_y1, pcg_unique_y1)
df_pcg_count_y2 = one_hot_count("MemberID", "PrimaryConditionGroup", df_claim_y2, pcg_unique_y2)
df_pcg_count_y12 = one_hot_count("MemberID", "PrimaryConditionGroup", df_claim_y12, pcg_unique_y12)

Processed: 0/76038
Processed: 10000/76038
Processed: 20000/76038
Processed: 30000/76038
Processed: 40000/76038
Processed: 50000/76038
Processed: 60000/76038
Processed: 70000/76038
Processed: 76038/76038 DONE
Processed: 0/71435
Processed: 10000/71435
Processed: 20000/71435
Processed: 30000/71435
Processed: 40000/71435
Processed: 50000/71435
Processed: 60000/71435
Processed: 70000/71435
Processed: 71435/71435 DONE
Processed: 0/95507
Processed: 10000/95507
Processed: 20000/95507
Processed: 30000/95507
Processed: 40000/95507
Processed: 50000/95507
Processed: 60000/95507
Processed: 70000/95507
Processed: 80000/95507
Processed: 90000/95507
Processed: 95507/95507 DONE
CPU times: user 15min 6s, sys: 14 s, total: 15min 20s
Wall time: 19min 31s


In [189]:
df_pcg_count_y12

,PrimaryConditionGroupCount_NEUMENT,PrimaryConditionGroupCount_FXDISLC,PrimaryConditionGroupCount_TRAUMA,PrimaryConditionGroupCount_RESPR4,PrimaryConditionGroupCount_INFEC4,PrimaryConditionGroupCount_ROAMI,PrimaryConditionGroupCount_HEART2,PrimaryConditionGroupCount_MISCHRT,PrimaryConditionGroupCount_PNEUM,PrimaryConditionGroupCount_MISCL5,...,PrimaryConditionGroupCount_HIPFX,PrimaryConditionGroupCount_GYNECA,PrimaryConditionGroupCount_CATAST,PrimaryConditionGroupCount_SEPSIS,PrimaryConditionGroupCount_LIVERDZ,PrimaryConditionGroupCount_PNCRDZ,PrimaryConditionGroupCount_METAB1,PrimaryConditionGroupCount_RENAL1,PrimaryConditionGroupCount_CANCRM,PrimaryConditionGroupCount_PERINTL
MemberID,,,,,,,,,,,,,,,,,,,,,
42286978,3,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11837054,0,15,0,3,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
99829076,0,0,12,1,0,10,0,14,0,1,...,0,0,0,0,0,0,0,0,0,0
54666321,0,0,2,48,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
60497718,0,0,4,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137916,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94826294,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3867712,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
df_pcg_count_y1.to_csv("./processed/PrimaryConditionGroup_Y1.csv")
df_pcg_count_y2.to_csv("./processed/PrimaryConditionGroup_Y2.csv")
df_pcg_count_y12.to_csv("./processed/PrimaryConditionGroup_Y12.csv")

## ProcedureGroup

In [190]:
pg_unique_y1 = list(df_claim_y1["ProcedureGroup"].unique())
pg_unique_y2 = list(df_claim_y2["ProcedureGroup"].unique())
pg_unique_y12 = list(df_claim_y12["ProcedureGroup"].unique())

In [191]:
%%time
df_pg_count_y1 = one_hot_count("MemberID", "ProcedureGroup", df_claim_y1, pg_unique_y1)
df_pg_count_y2 = one_hot_count("MemberID", "ProcedureGroup", df_claim_y2, pg_unique_y2)
df_pg_count_y12 = one_hot_count("MemberID", "ProcedureGroup", df_claim_y12, pg_unique_y12)

Processed: 0/76038
Processed: 10000/76038
Processed: 20000/76038
Processed: 30000/76038
Processed: 40000/76038
Processed: 50000/76038
Processed: 60000/76038
Processed: 70000/76038
Processed: 76038/76038 DONE
Processed: 0/71435
Processed: 10000/71435
Processed: 20000/71435
Processed: 30000/71435
Processed: 40000/71435
Processed: 50000/71435
Processed: 60000/71435
Processed: 70000/71435
Processed: 71435/71435 DONE
Processed: 0/95507
Processed: 10000/95507
Processed: 20000/95507
Processed: 30000/95507
Processed: 40000/95507
Processed: 50000/95507
Processed: 60000/95507
Processed: 70000/95507
Processed: 80000/95507
Processed: 90000/95507
Processed: 95507/95507 DONE
CPU times: user 8min 27s, sys: 8.8 s, total: 8min 36s
Wall time: 13min 1s


In [192]:
df_pg_count_y1.to_csv("./processed/ProcedureGroup_Y1.csv")
df_pg_count_y2.to_csv("./processed/ProcedureGroup_Y2.csv")
df_pg_count_y12.to_csv("./processed/ProcedureGroup_Y12.csv")

## LabCount

In [206]:
df_lab["LabCount"].replace({ "10+": 10 }, inplace=True)

TypeError: Cannot compare types 'ndarray(dtype=int32)' and 'str'

In [207]:
df_lab["LabCount"] = df_lab["LabCount"].astype("float64")

In [208]:
df_lab_y1 = df_lab[df_lab["Year"] == "Y1"]
print(df_lab_y1.shape)
df_lab_y2 = df_lab[df_lab["Year"] == "Y2"]
print(df_lab_y2.shape)
df_lab_y12 = df_lab[(df_lab["Year"] == "Y1") | (df_lab["Year"] == "Y2")]
print(df_lab_y12.shape)

(120162, 4)
(122416, 4)
(242578, 4)


In [209]:
def process_lab(df):
    lab_min = df.groupby(["MemberID"])["LabCount"].min()
    lab_max = df.groupby(["MemberID"])["LabCount"].max()
    lab_avg = df.groupby(["MemberID"])["LabCount"].mean()
    lab_std = df.groupby(["MemberID"])["LabCount"].std()
    lab_sum = df.groupby(["MemberID"])["LabCount"].sum()
    lab_claim_count = df.groupby(["MemberID"])["MemberID"].count()

    return pd.concat([
        pd.DataFrame(np.array(lab_min).T, columns=["LabCountMin"], index=lab_min.index),
        pd.DataFrame(np.array(lab_max).T, columns=["LabCountMax"], index=lab_max.index),
        pd.DataFrame(np.array(lab_avg).T, columns=["LabCountAvg"], index=lab_avg.index),
        pd.DataFrame(np.array(lab_std).T, columns=["LabCountStd"], index=lab_std.index),
        pd.DataFrame(np.array(lab_sum).T, columns=["LabCountSum"], index=lab_sum.index),
        pd.DataFrame(np.array(lab_claim_count).T, columns=["LabClaimCount"], index=lab_claim_count.index),
    ], axis=1)


In [211]:
%%time
df_lab_count_y1 = process_lab(df_lab_y1)
df_lab_count_y2 = process_lab(df_lab_y2)
df_lab_count_y12 = process_lab(df_lab_y12)

ok
ok
ok
CPU times: user 376 ms, sys: 30.4 ms, total: 407 ms
Wall time: 952 ms


In [213]:
df_lab_count_y1.to_csv("./processed/LabCount_Y1.csv")
df_lab_count_y2.to_csv("./processed/LabCount_Y2.csv")
df_lab_count_y12.to_csv("./processed/LabCount_Y12.csv")

## DrugCount

In [214]:
df_drug["DrugCount"].replace({ "7+": 7 }, inplace=True)

In [215]:
df_drug["DrugCount"] = df_drug["DrugCount"].astype("float64")

In [217]:
df_drug_y1 = df_drug[df_drug["Year"] == "Y1"]
print(df_drug_y1.shape)
df_drug_y2 = df_drug[df_drug["Year"] == "Y2"]
print(df_drug_y2.shape)
df_drug_y12 = df_drug[(df_drug["Year"] == "Y1") | (df_drug["Year"] == "Y2")]
print(df_drug_y12.shape)

(281619, 4)
(276027, 4)
(557646, 4)


In [218]:
def process_drug(df):
    drug_min = df.groupby(["MemberID"])["DrugCount"].min()
    drug_max = df.groupby(["MemberID"])["DrugCount"].max()
    drug_avg = df.groupby(["MemberID"])["DrugCount"].mean()
    drug_std = df.groupby(["MemberID"])["DrugCount"].std()
    drug_sum = df.groupby(["MemberID"])["DrugCount"].sum()
    drug_claim_count = df.groupby(["MemberID"])["DrugCount"].count()

    return pd.concat([
        pd.DataFrame(np.array(drug_min).T, columns=["DrugCountMin"], index=drug_min.index),
        pd.DataFrame(np.array(drug_max).T, columns=["DrugCountMax"], index=drug_max.index),
        pd.DataFrame(np.array(drug_avg).T, columns=["DrugCountAvg"], index=drug_avg.index),
        pd.DataFrame(np.array(drug_std).T, columns=["DrugCountStd"], index=drug_std.index),
        pd.DataFrame(np.array(drug_sum).T, columns=["DrugCountSum"], index=drug_sum.index),
        pd.DataFrame(np.array(drug_claim_count).T, columns=["DrugClaimCount"], index=drug_claim_count.index),
    ], axis=1)


In [219]:
%%time
df_drug_count_y1 = process_drug(df_drug_y1)
df_drug_count_y2 = process_drug(df_drug_y2)
df_drug_count_y12 = process_drug(df_drug_y12)

CPU times: user 519 ms, sys: 51.2 ms, total: 570 ms
Wall time: 788 ms


In [220]:
df_drug_count_y1.to_csv("./processed/DrugCount_Y1.csv")
df_drug_count_y2.to_csv("./processed/DrugCount_Y2.csv")
df_drug_count_y12.to_csv("./processed/DrugCount_Y12.csv")